<a href="https://colab.research.google.com/github/MedhaHegde66/medha-github-workshop/blob/main/IAI11_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def letter_check(letter, word):
    if letter in word:
        return True
    else:
        return False

# Taking input from the user
letter = input("Enter a letter: ")
word = input("Enter a word: ")

if letter_check(letter, word):
    print(f"The letter '{letter}' is present in the word '{word}'.")
else:
    print(f"The letter '{letter}' is not present in the word '{word}'.")


Enter a letter: i
Enter a word: sike
The letter 'i' is present in the word 'sike'.


In [2]:
import itertools

def distance(city1, city2):

    return ((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)**0.5

def total_distance(path, cities):
    total = 0
    for i in range(len(path) - 1):
        total += distance(cities[path[i]], cities[path[i + 1]])
    total += distance(cities[path[-1]], cities[path[0]])  # Return to starting city
    return total

def solve_tsp(cities):
    num_cities = len(cities)
    best_path = None
    min_distance = float('inf')

    for perm in itertools.permutations(range(num_cities)):
        current_distance = total_distance(perm, cities)
        if current_distance < min_distance:
            min_distance = current_distance
            best_path = perm

    return best_path, min_distance

# Example cities represented as (x, y) coordinates
cities = [(0, 0), (1, 3), (4, 6), (7, 1)]

best_path, min_distance = solve_tsp(cities)

print("Best path:", best_path)
print("Minimum distance:", min_distance)


Best path: (0, 1, 2, 3)
Minimum distance: 20.30693805399844


In [3]:
class WumpusWorld:
    def __init__(self, size):
        self.size = size
        self.grid = [[' ' for _ in range(size)] for _ in range(size)]
        self.agent_position = (0, 0)
        self.has_gold = False
        self.has_wumpus = False
        self.has_pit = False

    def move(self, direction):
        x, y = self.agent_position
        if direction == 'up' and x > 0:
            self.agent_position = (x - 1, y)
        elif direction == 'down' and x < self.size - 1:
            self.agent_position = (x + 1, y)
        elif direction == 'left' and y > 0:
            self.agent_position = (x, y - 1)
        elif direction == 'right' and y < self.size - 1:
            self.agent_position = (x, y + 1)

    def shoot(self):
        pass  # Implement shooting logic here

    def perceive(self):
        x, y = self.agent_position
        perceptions = []

        if self.grid[x][y] == 'W':
            perceptions.append('Stench')
        if self.grid[x][y] == 'P':
            perceptions.append('Breeze')
        if self.grid[x][y] == 'G':
            perceptions.append('Glitter')

        return perceptions

    def perform_action(self, action):
        if action == 'move':
            self.move(self.agent_position)
        elif action == 'shoot':
            self.shoot()

    def solve(self):
        visited = set()

        def dfs(position):
            if position in visited:
                return
            visited.add(position)

            # Perform actions based on perceptions
            perceptions = self.perceive()
            if 'Glitter' in perceptions:
                self.has_gold = True
            if 'Stench' in perceptions:
                self.has_wumpus = True
            if 'Breeze' in perceptions:
                self.has_pit = True

            # Recursive DFS for neighboring cells
            for direction in ['up', 'down', 'left', 'right']:
                self.move(direction)
                next_position = self.agent_position
                if next_position not in visited:
                    dfs(next_position)
                self.move(direction)  # Move back

        dfs(self.agent_position)


# Create a Wumpus World of size 4x4
world = WumpusWorld(size=4)

# Initialize the grid with objects (Wumpus, Pit, Gold)
world.grid[1][1] = 'W'
world.grid[2][2] = 'P'
world.grid[3][3] = 'G'

# Solve the Wumpus World problem
world.solve()

# Print the agent's final state and perceptions
print("Agent's final position:", world.agent_position)
print("Has Gold:", world.has_gold)
print("Has Wumpus:", world.has_wumpus)
print("Has Pit:", world.has_pit)


Agent's final position: (3, 3)
Has Gold: True
Has Wumpus: True
Has Pit: True


In [4]:
import heapq

class PuzzleNode:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = 0 if parent is None else parent.cost + 1

    def __lt__(self, other):
        return (self.cost + self.heuristic()) < (other.cost + other.heuristic())

    def heuristic(self):
        # Manhattan distance heuristic
        total_distance = 0
        for i in range(3):
            for j in range(3):
                value = self.state[i][j]
                if value != 0:
                    goal_i, goal_j = divmod(value - 1, 3)
                    total_distance += abs(i - goal_i) + abs(j - goal_j)
        return total_distance

    def generate_children(self):
        children = []
        i, j = next((i, j) for i in range(3) for j in range(3) if self.state[i][j] == 0)
        for action, (di, dj) in [('up', (-1, 0)), ('down', (1, 0)), ('left', (0, -1)), ('right', (0, 1))]:
            ni, nj = i + di, j + dj
            if 0 <= ni < 3 and 0 <= nj < 3:
                new_state = [row[:] for row in self.state]
                new_state[i][j], new_state[ni][nj] = new_state[ni][nj], new_state[i][j]
                children.append(PuzzleNode(new_state, parent=self, action=action))
        return children

    def get_solution(self):
        solution = []
        node = self
        while node is not None:
            if node.action:
                solution.append(node.action)
            node = node.parent
        solution.reverse()
        return solution

def solve_8_puzzle(initial_state, goal_state):
    initial_node = PuzzleNode(initial_state)
    open_list = [initial_node]
    closed_set = set()

    while open_list:
        current_node = heapq.heappop(open_list)
        if current_node.state == goal_state:
            return current_node.get_solution()

        closed_set.add(tuple(map(tuple, current_node.state)))
        children = current_node.generate_children()
        for child in children:
            if tuple(map(tuple, child.state)) not in closed_set:
                heapq.heappush(open_list, child)

    return None

# Example initial and goal states
initial_state = [[1, 2, 3], [4, 5, 0], [7, 8, 6]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# Solve the 8-puzzle problem
solution = solve_8_puzzle(initial_state, goal_state)

if solution:
    print("Solution:", solution)
else:
    print("No solution found.")


Solution: ['down']


In [5]:
class KnowledgeBase:
    def __init__(self):
        self.facts = set()

    def tell(self, fact):
        self.facts.add(fact)

    def ask(self, query):
        return query in self.facts


def forward_chaining(kb, rules, target):
    inferred = set()

    while True:
        new_inferred = set()
        for rule in rules:
            premise, conclusion = rule
            if all(fact in inferred for fact in premise) and conclusion not in inferred:
                new_inferred.add(conclusion)

        if not new_inferred:
            break

        inferred.update(new_inferred)

    return target in inferred


def backward_chaining(kb, rules, target):
    def bc(query):
        if query in kb.facts:
            return True

        for rule in rules:
            premise, conclusion = rule
            if conclusion == query:
                if all(bc(p) for p in premise):
                    return True
        return False

    return bc(target)


if __name__ == "__main__":
    # Initialize knowledge base
    kb = KnowledgeBase()

    # Add facts to the knowledge base
    kb.tell("learner's permit")
    kb.tell("age >= 18")

    # Define rules (premises, conclusion)
    rules = [
        (["age >= 18", "learner's permit"], "eligible for driver's license")
    ]

    # Perform forward chaining and backward chaining
    forward_result = forward_chaining(kb, rules, "eligible for driver's license")
    backward_result = backward_chaining(kb, rules, "eligible for driver's license")

    # Print results
    print("Forward Chaining Result:", forward_result)
    print("Backward Chaining Result:", backward_result)


Forward Chaining Result: False
Backward Chaining Result: True
